### Module 8: Demo8 Regularization(正則化)

In [ ]:
import torch
from torchvision.datasets import mnist
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
def data_transform(x):
    x = np.array(x, dtype = 'float32') / 255
    x = x.reshape((-1, )); x = torch.from_numpy(x); return x
trainset = mnist.MNIST('./dataset/mnist', train=True, transform=data_transform, download=True)
testset = mnist.MNIST('./dataset/mnist', train = False, transform=data_transform, download=True)
train_data = DataLoader(trainset, batch_size=64, shuffle=True)
test_data = DataLoader(testset, batch_size=128, shuffle=False)
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 500); self.fc2 = nn.Linear(500, 250)
        self.fc3 = nn.Linear(250, 125);   self.fc4 = nn.Linear(125, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x)); x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x)); x = self.fc4(x) ; return x
model = DNN().to(device) # build model
criterion = nn.CrossEntropyLoss()  # define cost
# add L2 regularization 
optimizer = torch.optim.SGD(model.parameters(), 0.1, weight_decay=1e-5)
#optimizer = torch.optim.RMSprop(model.parameters(), 0.01, alpha=0.9)
#optimizer = torch.optim.Adam(model.parameters(), 0.01, betas=(0.9, 0.99))
losses = [];    acces = []
eval_losses = [] ;  eval_acces = []
for epoch in range(5):
    train_loss = 0 ; train_acc = 0
    model.train()       
    for im, label in train_data:
        im = im.to(device)  ;label = label.to(device)
        pred = model(im)
        loss = criterion(pred, label)
        optimizer.zero_grad()   # zero the parameter gradients
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, pred = pred.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        train_acc += acc
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    eval_loss = 0 ;   val_acc = 0
    model.eval()
    for im, label in test_data:
        im = im.to(device) ; label = label.to(device)
        pred = model(im)
        loss = criterion(pred, label)
        eval_loss += loss.item()
        _, pred = pred.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        eval_acc += acc
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}, Eval Loss: {:.6f}, Eval Acc: {:.6f}'
          .format(epoch, train_loss / len(train_data), train_acc / len(train_data), eval_loss / len(test_data), eval_acc / len(test_data)))

# Dropout

In [ ]:
import torch
from torchvision.datasets import mnist
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
def data_transform(x):
    x = np.array(x, dtype = 'float32') / 255
    x = x.reshape((-1, ))
    x = torch.from_numpy(x)
    return x
trainset = mnist.MNIST('./dataset/mnist', train=True, transform=data_transform, download=True)
testset = mnist.MNIST('./dataset/mnist', train = False, transform=data_transform, download=True)
train_data = DataLoader(trainset, batch_size=64, shuffle=True)
test_data = DataLoader(testset, batch_size=128, shuffle=False)
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 250)
        self.fc3 = nn.Linear(250, 125)
        self.fc4 = nn.Linear(125, 10)
        # Define proportion or neurons to dropout
        self.dropout = nn.Dropout(0.25)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x
model = DNN().to(device) # build model
criterion = nn.CrossEntropyLoss()  # define cost
# optimization
optimizer = torch.optim.SGD(model.parameters(), 0.1)
#optimizer = torch.optim.RMSprop(model.parameters(), 0.01, alpha=0.9)
#optimizer = torch.optim.Adam(model.parameters(), 0.01, betas=(0.9, 0.99))
losses = [] ;acces = []
eval_losses = [];eval_acces = []
for epoch in range(5):
    train_loss = 0  ;     train_acc = 0
    model.train()       
    for im, label in train_data:
        im = im.to(device)
        label = label.to(device)
        pred = model(im)
        loss = criterion(pred, label)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, pred = pred.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        train_acc += acc
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    eval_loss = 0
    eval_acc = 0
    model.eval()
    for im, label in test_data:
        im = im.to(device)
        label = label.to(device)
        pred = model(im)
        loss = criterion(pred, label)
        eval_loss += loss.item()
        _, pred = pred.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        eval_acc += acc
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}, Eval Loss: {:.6f}, Eval Acc: {:.6f}'
          .format(epoch, train_loss / len(train_data), train_acc / len(train_data), eval_loss / len(test_data), eval_acc / len(test_data)))

# Batch Normalization

In [ ]:
import torch
from torchvision.datasets import mnist
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
def data_transform(x):
    x = np.array(x, dtype = 'float32') / 255
    x = x.reshape((-1, ))
    x = torch.from_numpy(x)
    return x
trainset = mnist.MNIST('./dataset/mnist', train=True, transform=data_transform, download=True)
testset = mnist.MNIST('./dataset/mnist', train = False, transform=data_transform, download=True)
train_data = DataLoader(trainset, batch_size=64, shuffle=True)
test_data = DataLoader(testset, batch_size=128, shuffle=False)
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.bn1 = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 250)
        self.bn2 = nn.BatchNorm1d(250)
        self.fc3 = nn.Linear(250, 125)
        self.bn3 = nn.BatchNorm1d(125)
        self.fc4 = nn.Linear(125, 10)
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = self.bn3(x)
        x = F.relu(x)        
        x = self.fc4(x)
        return x
model = DNN().to(device) # build model
criterion = nn.CrossEntropyLoss()  # define cost
# optimization
optimizer = torch.optim.SGD(model.parameters(), 0.1)
#optimizer = torch.optim.RMSprop(model.parameters(), 0.01, alpha=0.9)
#optimizer = torch.optim.Adam(model.parameters(), 0.01, betas=(0.9, 0.99))
losses = [] ;acces = []
eval_losses = []  ;eval_acces = []
for epoch in range(5):
    train_loss = 0
    train_acc = 0
    model.train()       
    for im, label in train_data:
        im = im.to(device)
        label = label.to(device)
        pred = model(im)
        loss = criterion(pred, label)
        optimizer.zero_grad()  # zero the parameter gradients
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, pred = pred.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        train_acc += acc
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    eval_loss = 0
    eval_acc = 0
    model.eval()
    for im, label in test_data:
        im = im.to(device)
        label = label.to(device)
        pred = model(im)
        loss = criterion(pred, label)
        eval_loss += loss.item()
        _, pred = pred.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        eval_acc += acc
       eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}, Eval Loss: {:.6f}, Eval Acc: {:.6f}'
          .format(epoch, train_loss / len(train_data), train_acc / len(train_data), eval_loss / len(test_data), eval_acc / len(test_data)))